In [1]:
import os, glob, json 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sys 
sys.path.append('./helper')

In [3]:
# point the example data 
data_dir = glob.glob('/mnt/HDD/jiwon/2024_spring_pilot/sub105/sub105_walking+memory*')[0]

### Calibrate gaze points

In [10]:
# load transformation parameters and transform data 
import skimage.transform as tf

saving_dir = data_dir + '/gaze_calibrated.csv'
# if the gaze_calibrated.csv is existing, skip this part 
if not os.path.exists(saving_dir): 
    params_dir = os.path.dirname(data_dir) + '/gaze_calibration_params.json'
    with open(params_dir, 'r') as file:
        params = json.load(file)

    gazes = pd.read_csv(data_dir + '/gaze.csv')

    # convert gazes in visual angle 
    matrix_deg = tf.AffineTransform(matrix = params['tf_matrix_deg'])
    transformed = matrix_deg(gazes[['azimuth [deg]', 'elevation [deg]']].values)
    gazes['gaze x [deg]'] = transformed[:,0]
    gazes['gaze y [deg]'] = transformed[:,1]

    # drop azimuth and elevation 
    gazes.drop(columns = ['azimuth [deg]', 'elevation [deg]'])

    # convert gazes in px 
    matrix_px = tf.AffineTransform(matrix = params['tf_matrix_px'])
    transformed = matrix_px(gazes[['gaze x [px]', 'gaze y [px]']].values)
    gazes['gaze x [px]'] = transformed[:,0]
    gazes['gaze y [px]'] = transformed[:,1]

    # save gazes 
    gazes.to_csv(saving_dir)

else:
    print(f'{os.path.basename(saving_dir)} is existing already. Load the data...')
    gazes = pd.read_csv(saving_dir)


gaze_calibrated.csv is existing already. Skipping this procedure.


### Exclude gaze points that are not saccade or blinks

In [13]:
# create blink mask 
blinks = pd.read_csv(data_dir +'/blinks.csv')
is_blink = np.zeros(len(gazes), dtype=bool)

for b in range(len(blinks)):
    start_time = blinks.loc[b, 'start timestamp [ns]']
    end_time = blinks.loc[b, 'end timestamp [ns]']
    
    # find timestamps that fall within this blink's time range
    is_in_blink = (gazes['timestamp [ns]'] >= start_time) & (gazes['timestamp [ns]'] <= end_time)
    
    # set mask to false for timestmpas during blinks 
    is_blink[is_in_blink] = True
    


In [ ]:
# create main_sequence mask 


### Compute world gaze and gaze angle

In [13]:
import gaze_angle_relative_to_gravity as gg 

root_dir = glob.glob('/mnt/HDD/jiwon/2024_spring_pilot/sub105/*walking+memory*')[0] + '/'
gaze_file = 'gaze.csv'

world_gaze = gg.get_world_gaze(root_dir, gaze_file)


In [17]:
angle_ = gg.get_gaze_angle_relative_to_gravity(world_gaze)

### Use event file to segment the gaze angle data by the locations